Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## Define Strategy

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

In [ ]:
class SimpleStrategy(Strategy):
    def __init__(self, asset, quantity):
        super().__init__()
        self.asset = asset
        self.quantity = quantity

    def log_all(self, orders, data, ctx, utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(utc)
            self.log_ohlcv(data, self.asset.symbol, ctx.exchange.id)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        orders = []
        quantity = .05
        price = data.get('close', self.asset.symbol, ctx.exchange.id)
        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, self.asset, quantity, price)
            orders.append(order)
        else:
            order = order_manager.build_market_sell_order(
                ctx.exchange, self.asset, quantity)
            orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []

        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)
        print(data.get('utc'))
        self.log_all(orders, data, ctx, data.get('utc'))
        return {
            'orders': orders,
            'cancel_ids': cancel_ids
        }

## Backtest

In [ ]:
cash = 1.0
ohlcv_fpath = os.path.join(cfg.DATA_DIR, c.DEFAULT_30M_FEED_CSV_FILENAME)
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.THIRTY_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = OHLCVFileFeed(
    fpath=ohlcv_fpath,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(balance, feed)
asset = Asset(c.ETH, c.BTC)
strategy = SimpleStrategy(asset, quantity=.05)
experiment_name = 'default_backtest'

In [ ]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

## Simulate Orders

In [ ]:
cash = 1.0
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

assets = [Asset(c.ETH, c.BTC), Asset(c.LTC, c.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow()-datetime.timedelta(days=1),
    end=None
)
strategy = SimpleStrategy(assets[0], quantity=.05)

In [ ]:
experiment_name = 'default_backtest_exchange'
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

In [ ]:
experiment_name = 'default_simulate'
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

## Live Trade

In [ ]:
exchange = load_exchange(c.BINANCE)
cash = exchange.fetch_balance().get(c.BTC)[BalanceType.FREE]
print(c.BTC, "cash", cash)

In [ ]:
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)

In [ ]:
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
asset = Asset(c.ETH, c.BTC)
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=[asset],
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy(asset, quantity=.05)
experiment_name = 'default_live'

In [ ]:
runner.live(experiment_name, exchange, balance, portfolio, feed, strategy)